### Container with Most Water

In [4]:
class Solution(object):
    def maxArea(self, height):
        left = 0
        right = len(height) - 1
        max_area = 0

        while left < right:
            width = right - left
            min_height = min(height[left], height[right])
            area = width * min_height
            max_area = max(max_area, area)
            if height[left] < height[right]:
                left += 1
            else:
                right -= 1
        return max_area

s = Solution()
print(s.maxArea([1,8,6,2,5,4,8,3,7]))  # Output: 49
print(s.maxArea([1,1]))               # Output: 1



49
1


### Full Application

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Define a diverse set of tickers
tickers = ['AAPL', 'TSLA', 'GOOG', 'NVDA']

# Download adjusted close prices from 2022
prices = yf.download(tickers, start="2022-01-01", end="2023-01-01")['Adj Close']
returns = prices.pct_change().dropna()

# Compute rolling Sharpe-like signal: mean / std
sharpe_signal = returns.rolling(21).mean() / returns.rolling(21).std()
sharpe_signal = sharpe_signal.dropna()

def find_optimal_span_band(signal_array):
    """
    Use two-pointer technique to find widest pair with high min confidence (like max water container).
    """
    left = 0
    right = len(signal_array) - 1
    max_score = 0
    best_pair = (0, 0)

    while left < right:
        width = right - left
        height = min(signal_array[left], signal_array[right])
        score = width * height

        if score > max_score:
            max_score = score
            best_pair = (left, right)

        if signal_array[left] < signal_array[right]:
            left += 1
        else:
            right -= 1

    return best_pair, max_score

asset = 'AAPL'
signal_series = sharpe_signal[asset].fillna(0).values
(pair_start, pair_end), max_area = find_optimal_span_band(signal_series)

# Plot
dates = sharpe_signal.index
plt.figure(figsize=(12, 6))
plt.plot(dates, sharpe_signal[asset], label='21-Day Sharpe-like Signal')
plt.axvline(dates[pair_start], color='green', linestyle='--', label='Start Anchor')
plt.axvline(dates[pair_end], color='red', linestyle='--', label='End Anchor')
plt.fill_between(dates[pair_start:pair_end+1],
                 sharpe_signal[asset].iloc[pair_start:pair_end+1].min(),
                 sharpe_signal[asset].iloc[pair_start:pair_end+1].min(),
                 color='blue', alpha=0.3, label='Risk-Min Span')
plt.title(f'Optimal Risk-Bounded Spread for {asset} | Score = {max_area:.2f}')
plt.xlabel('Date')
plt.ylabel('Sharpe Signal')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()




